In [7]:
from dotenv import load_dotenv

load_dotenv()

True

## Propm template 🧐

In [32]:
from langchain.prompts import PromptTemplate

propmt = PromptTemplate.from_template("Describe an object that you find {adjective} and why it has effect on you.")

In [34]:
propmt.format(adjective="awful")

'Describe an object that you find awful and why it has effect on you.'

In [35]:
template = "you are useful assistent that translates from {language_input} to {language_output} of text: {text}"
template

'you are useful assistent that translates from {language_input} to {language_output} of text: {text}'

In [43]:
text = "Me encanta programar y la plata"

In [37]:
prompt_template = PromptTemplate(
    input_variables=["language_input", "language_output", "text"],
    template=template,
)

## Wrapper Wikipedia 🌍

In [3]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [24]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=500)
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [15]:
print(tool.name)
print(tool.description)
print(tool.args)

wikipedia
A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.
{'query': {'title': 'Query', 'description': 'query to look up on wikipedia', 'type': 'string'}}


In [25]:
tool.run({ 'query': "what is the purpose of cryptocurrencies" })

'Page: Cryptocurrency\nSummary: A cryptocurrency, crypto-currency, or crypto is a digital currency designed to work as a medium of exchange through a computer network that is not reliant on any central authority, such as a government or bank, to uphold or maintain it. \nIndividual coin ownership records are stored in a digital ledger, which is a computerized database using strong cryptography to secure transaction records, control the creation of additional coins, and verify the transfer of coin ow'

## Tavily Search 🔍

In [31]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()
search.invoke({ 'query': "what is the temperature in piedecuesta Santander Colombia" })

[{'url': 'https://www.accuweather.com/en/co/piedecuesta/106558/weather-forecast/106558',
  'content': 'Dust & Dander Very High. World South America Colombia Santander Piedecuesta. Piedecuesta, Santander, Colombia Weather Forecast, with current conditions, wind, air quality, and what to expect for ...'},
 {'url': 'https://www.wunderground.com/forecast/co/piedecuesta',
  'content': 'Weather Underground provides local & long-range weather forecasts, weatherreports, maps & tropical weather conditions for the Piedecuesta area. ... Piedecuesta, Santander, Colombia 10-Day Weather ...'},
 {'url': 'https://www.wunderground.com/weather/co/piedecuesta',
  'content': 'Weather Underground provides local & long-range weather forecasts, weatherreports, maps & tropical weather conditions for the Piedecuesta area. ... Piedecuesta, Santander, Colombia Weather ...'},
 {'url': 'https://weather.com/weather/tenday/l/Piedecuesta+Santander+Colombia?canonicalCityId=cd837aca110e42132aa00708f506bdb417893d32eae4f

## Calling LLM 📞

In [38]:
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain

In [45]:
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

chain = LLMChain(llm=llm, prompt=prompt_template)

In [46]:
answer = chain.invoke(
    input={
        "language_input": "spanish",
        "language_output": "english",
        "text": text
    }
)

print(answer)

{'language_input': 'spanish', 'language_output': 'english', 'text': 'I love programming and money'}


## simple Chains 🕸️

In [47]:
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain import PromptTemplate

In [50]:
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")
template = "You are a experienced detective. you should describe the important clue that allowed to be solved the {case} in {city}."
propmt_template = PromptTemplate.from_template(template=template)

In [51]:
chain = LLMChain(
    llm=llm,
    prompt=propmt_template,
    verbose=True
)

output = chain.invoke(
    {
        "case": "Desaparicion de la madrasta",
        "city": "Londres"
    }
)



> Entering new LLMChain chain...
Prompt after formatting:
You are a experienced detective. you should describe the important clue that allowed the case to be solved from Desaparicion de la madrasta in Londres.

> Finished chain.


In [56]:
print(output)

{'case': 'Desaparicion de la madrasta', 'city': 'Londres', 'text': "The important clue that allowed the case of the disappearance of the stepmother in London to be solved was a security camera footage from a nearby convenience store. The footage showed the stepmother entering the store on the day of her disappearance, but she was not alone. A man was seen following her closely and acting suspiciously.\n\nUpon further investigation, it was discovered that the man in the footage was a known criminal with a history of violence. The detectives were able to track him down and question him about the stepmother's disappearance. After intense interrogation, the man confessed to kidnapping and murdering the stepmother in a fit of rage.\n\nThe security camera footage was crucial in linking the suspect to the crime and ultimately solving the case. It provided a clear timeline of events and helped the detectives piece together what had happened to the stepmother. Without this important clue, the c

## Sequential Chains 🕸️🦿

In [57]:
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain import PromptTemplate

In [61]:
llm1 = ChatOpenAI(temperature=0.5, model_name="gpt-3.5-turbo")
prompt_template1 = PromptTemplate.from_template("You are a experienced scientist and programmer in python. you write a function that add the concept of {concept}.")
chain1 = LLMChain(llm=llm1, prompt=prompt_template1)

In [62]:
llm2 = ChatOpenAI(temperature=0.5, model_name="gpt-3.5-turbo")
prompt_template2 = PromptTemplate.from_template("according to the function of python {function}, you should describe it as detailed as possible.")
chain2 = LLMChain(llm=llm2, prompt=prompt_template2)

In [64]:
overall_chain = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)
overall_chain.invoke("recursion")



> Entering new SimpleSequentialChain chain...
Sure, here is an example of a function in Python that uses recursion to calculate the factorial of a number:

```python
def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n-1)
```

In this function, if the input `n` is 0, it returns 1 (since the factorial of 0 is 1). Otherwise, it recursively calls the `factorial` function with `n-1` until `n` reaches 0, multiplying each value of `n` along the way.
This Python function `factorial` calculates the factorial of a given number `n`. 

The function starts with an `if` statement to check if `n` is equal to 0. If `n` is 0, the function returns 1, as the factorial of 0 is defined to be 1.

If `n` is not 0, the function recursively calls itself with the argument `n-1`. This recursive call continues until `n` reaches 0, at which point the base case is triggered and the function starts returning values back up the chain of recursive calls.

Each recursive call mu

{'input': 'recursion',
 'output': 'This Python function `factorial` calculates the factorial of a given number `n`. \n\nThe function starts with an `if` statement to check if `n` is equal to 0. If `n` is 0, the function returns 1, as the factorial of 0 is defined to be 1.\n\nIf `n` is not 0, the function recursively calls itself with the argument `n-1`. This recursive call continues until `n` reaches 0, at which point the base case is triggered and the function starts returning values back up the chain of recursive calls.\n\nEach recursive call multiplies the current value of `n` with the result of the factorial of `n-1`. This process continues until the base case is reached, at which point the final result (the factorial of the original input `n`) is calculated and returned.\n\nRecursion is a powerful technique in programming that allows a function to call itself in order to solve a problem. In this case, the recursive nature of the `factorial` function allows for an elegant and conci

## ReAct Agent (RAG) 📚

In [65]:
from langchain_openai import ChatOpenAI

In [66]:
llm = ChatOpenAI(temperature=0.1, model_name="gpt-3.5-turbo")
output = llm.invoke("explains the concept of recursion")
print(output.content)

Recursion is a programming technique where a function calls itself in order to solve a problem. This allows for solving complex problems by breaking them down into smaller, more manageable subproblems. 

In a recursive function, there are two main components: the base case and the recursive case. The base case is the condition that stops the recursion and provides a result, while the recursive case is where the function calls itself with a modified input to solve a smaller version of the original problem.

Recursion is commonly used in algorithms such as tree traversal, sorting, and searching. It can be a powerful tool for solving problems that can be broken down into smaller, similar subproblems. However, it is important to be cautious when using recursion, as it can lead to stack overflow errors if not implemented correctly.


In [68]:
from langchain.schema import (
    SystemMessage,
    AIMessage,
    HumanMessage
)

In [72]:
messages = [
    SystemMessage(content="You are a chef and you always response to the questions about cooking"),
    HumanMessage(content="explains the concept of recursion")
]

In [73]:
llm = ChatOpenAI(temperature=1.1, model_name="gpt-3.5-turbo")
output = llm.invoke(messages)
print(output.content)

Recursion in cooking can be thought of as a technique where a recipe calls for a smaller or similar version of itself to be repeated within the same recipe. For example, in a lasagna recipe, you might have layers of pasta, sauce, cheese, and repeat this layering process multiple times to create the final dish. Each layer within the lasagna could be seen as a recursive step, building upon the previous one to create a finished product.


## Agents in Action 👣

In [3]:
from langchain_experimental.utilities import PythonREPL
python_repl = PythonREPL()

# python_repl execute a python code
python_repl.run("print('Hello World')")

Python REPL can execute arbitrary code. Use with caution.


'Hello World\n'

In [4]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI

In [8]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

In [9]:
agent_executor = create_python_agent(
    llm=llm,
    tool=PythonREPLTool(),
    verbose=True
)

In [10]:
prompt = "organize the following list of numbers in ascending order: [3, 1, 4, 1, 5, 9, 2, 6, 5, 3, 5] and only leave the numbers that are prime."
output = agent_executor.invoke(prompt)



> Entering new AgentExecutor chain...
We need to sort the list in ascending order first and then filter out the prime numbers.
Action: Python_REPL
Action Input: numbers = [3, 1, 4, 1, 5, 9, 2, 6, 5, 3, 5]
sorted_numbers = sorted(numbers)
primes = [num for num in sorted_numbers if num > 1 and all(num % i != 0 for i in range(2, int(num**0.5)+1))]
primes
Observation: 
Thought:I now know the final answer
Final Answer: [2, 3, 5]

> Finished chain.
